# Feature extraction

In this notebook we will learn how to extract different features from a text and how to combine them. It's pretty simple, but if you have this part well organized, it will be really useful in the near future. So, let's get started!

In [2]:
import nltk
from sklearn.pipeline import FeatureUnion
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn import preprocessing
from scipy.sparse import coo_matrix, hstack
from copy import deepcopy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import glob
import json
import argparse
import time
import codecs
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

In [4]:
import csv
import re
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV




Using TensorFlow backend.


In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import os

In [8]:
def open_file(path):
    with open(path, 'r+', encoding='utf8') as f:
        return '\n'.join([line.strip() for line in f])
    
def process_dir_files(path):
    dir_files = []
    for file in os.listdir(path):
        current = os.path.join(path, file)
        if os.path.isfile(current):
            dir_files.append(open_file(current))
    return dir_files
                     

#train_sents= process_dir_files('pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02/problem00001/candidate00001')



In [9]:
from nltk.tokenize import word_tokenize

def get_pos(text):
    pos_tags= nltk.pos_tag(word_tokenize(text))
    # print(len(pos_tags))
    pos_tags = [word_tag[1] for word_tag in pos_tags]
    pos_text = ' '.join(pos_tags)
    return pos_tags

#get_pos(train_sents[0])

In [10]:
def get_pos_ngrams(sents):
    pos_tags= [nltk.pos_tag(word_tokenize(sents[ind])) for ind, item in enumerate(sents) if item != '']
    pos_sents = []
    for sent in pos_tags:
        #print(sent)
        pos = ' '.join([pos_tag[1] for pos_tag in sent])
        #print(pos, '\n')
        pos_sents.append(pos)
    vectorizer = CountVectorizer(ngram_range = (1,1))
    vectorizer.fit(pos_sents)
    return vectorizer


#pos_vectorizer = get_pos_ngrams(train_sents)
#pos_ngram = pos_vectorizer.transform(train_sents)
#pos_ngram

In [11]:
# -*- coding: utf-8 -*-

"""
 A baseline authorship attribution method 
 based on a character n-gram representation
 and a linear SVM classifier.
 It has a reject option to leave documents unattributed
 (when the probabilities of the two most likely training classes are too close)
 
 Questions/comments: stamatatos@aegean.gr

 It can be applied to datasets of PAN-19 cross-domain authorship attribution task
 See details here: http://pan.webis.de/clef19/pan19-web/author-identification.html
 Dependencies:
 - Python 2.7 or 3.6 (we recommend the Anaconda Python distribution)
 - scikit-learn

 Usage from command line: 
    > python pan19-cdaa-baseline.py -i EVALUATION-DIRECTORY -o OUTPUT-DIRECTORY [-n N-GRAM-ORDER] [-ft FREQUENCY-THRESHOLD] [-pt PROBABILITY-THRESHOLD]
 EVALUATION-DIRECTORY (str) is the main folder of a PAN-19 collection of attribution problems
 OUTPUT-DIRECTORY (str) is an existing folder where the predictions are saved in the PAN-19 format
 Optional parameters of the model:
   N-GRAM-ORDER (int) is the length of character n-grams (default=3)
   FREQUENCY-THRESHOLD (int) is the cutoff threshold used to filter out rare n-grams (default=5)
   PROBABILITY-THRESHOLD (float) is the threshold for the reject option assigning test documents to the <UNK> class (default=0.1)
                                 Let P1 and P2 be the two maximum probabilities of training classes for a test document. If P1-P2<pt then the test document is assigned to the <UNK> class.
   
 Example:

     >  python pan19-cdaa-baseline-svm.py -i ".\pan19-cross-domain-authorship-attribution-training-dataset-2019-01-23\" -o ".\a
nswers-trigram\" -n 3
"""

from __future__ import print_function
import os
import glob
import json
import argparse
import time
import codecs
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.calibration import CalibratedClassifierCV

def represent_text(text,n,pos=False):
    # Extracts all character n-grams from  a 'text'
    # if pos is True, extracts POS n-grams
    if n>0:
        if pos is True:
            text = get_pos(text)
            tokens = [' '.join(text[i:i+n]) for i in range(len(text)-n+1)]
            #print(tokens)
        else:
            tokens = [text[i:i+n] for i in range(len(text)-n+1)]
    frequency = defaultdict(int)
    for token in tokens:
        frequency[token] += 1
    return frequency

#represent_text(train_sents[0], 2)
#represent_text(train_sents[0], 2, pos=True)
                           

In [12]:
def read_files(path,label):
    # Reads all text files located in the 'path' and assigns them to 'label' class
    files = glob.glob(path+os.sep+label+os.sep+'*.txt')
    texts=[]
    for i,v in enumerate(files):
        f=codecs.open(v,'r',encoding='utf-8')
        texts.append((f.read(),label))
        f.close()
    return texts

def extract_vocabulary(texts,n,ft,pos=False):
    # Extracts all characer 'n'-grams occurring at least 'ft' times in a set of 'texts'
    occurrences=defaultdict(int) 
    for (text,label) in texts:
        text_occurrences = {}
        if isinstance(n, int):
            for x in range(1,n+1):
                text_occurrences.update(represent_text(text,x,pos=pos))
        else:
            pass
        for ngram in text_occurrences:
            if ngram in occurrences:
                occurrences[ngram]+=text_occurrences[ngram]
            else:
                occurrences[ngram]=text_occurrences[ngram]
    vocabulary=[]
    for i in occurrences.keys():
        if occurrences[i]>=ft:
            vocabulary.append(i)
    return vocabulary

'''
vocab = extract_vocabulary([(x,i) for i, x in enumerate(train_sents)], 2, 5, pos=True)
print(len(vocab))
print(vocab)
vectorizer = CountVectorizer(vocabulary=[x.lower() for  x in vocab])
print([' '.join(get_pos(text)) for text in train_sents])

train_data = vectorizer.fit_transform([' '.join(get_pos(text)) for text in train_sents])
print(vectorizer.get_feature_names())
train_data = train_data.astype(float)
print(train_data.shape)
print(train_data.toarray())
'''

"\nvocab = extract_vocabulary([(x,i) for i, x in enumerate(train_sents)], 2, 5, pos=True)\nprint(len(vocab))\nprint(vocab)\nvectorizer = CountVectorizer(vocabulary=[x.lower() for  x in vocab])\nprint([' '.join(get_pos(text)) for text in train_sents])\n\ntrain_data = vectorizer.fit_transform([' '.join(get_pos(text)) for text in train_sents])\nprint(vectorizer.get_feature_names())\ntrain_data = train_data.astype(float)\nprint(train_data.shape)\nprint(train_data.toarray())\n"

In [13]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [49]:
STOPWORDS = list(stopwords.words('english'))
len(STOPWORDS)

179

In [15]:
lex_div = np.zeros((5,1))
lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
lex_div

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [16]:
from sklearn.base import TransformerMixin #gives fit_transform method for free

class Feature_Extractor(TransformerMixin):
    '''
    Performs feature extraction on input docs.
    '''
    def __init__(self, char=True, pos=True, word=True, lexdiv=True, fnword=True):
        self.char = char # uses character ngrams
        self.pos = pos # uses pos ngrams
        self.word = word # uses word ngrams
        self.fnword = fnword # uses function word counts
        self.lexdiv = lexdiv # uses lexical diversity score
        
    def set_params(self, char, pos, word, lexdiv, fnword):
        self.char = char # uses character ngrams
        self.pos = pos # uses pos ngrams
        self.word = word # uses word ngrams
        self.fnword = fnword # uses function word counts
        self.lexdiv = lexdiv # uses lexical diversity score

        
    def fit_transform(self, docs, y=None):
        ## Char-level n-grams ##
        feature_list = []
        self.texts = [text for i,(text,label) in enumerate(docs)]
        print('Extracting features from train data')
        if self.lexdiv is True or not any((self.char, self.pos, self.word, self.fnword, self.lexdiv)) :
            ## Lexical Diversity
            lex_div = self.lexical_diversity(docs)
            if not any((self.char, self.pos, self.word, self.fnword)):
                lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
            feature_list.append(lex_div)
            
        if self.char is True:
            char_vocab = extract_vocabulary(docs, 4, 3)
            self.char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(4,4),
                                              lowercase=False, vocabulary=char_vocab
                                                  )
            char_data, self.char_vectorizer = self._fit_transform(self.char_vectorizer)
            print('char vocabulary size:', len(char_vocab))
            feature_list.append(char_data)
            # print(char_data.toarray())
        
        ## POS n-grams ##
        if self.pos is True:
            pos_vocab = [x.lower() for x in extract_vocabulary(docs,2,3,pos=True)]
            # print(pos_vocab)
            self.pos_vectorizer = CountVectorizer(ngram_range=(1,3), vocabulary=pos_vocab
                                                 )
            print('\t', 'pos vocabulary size:', len(pos_vocab))
            pos_data, self.pos_vectorizer = self._fit_transform(self.pos_vectorizer, pos_replace=True)
            feature_list.append(pos_data)

        ## Word n-grams ##
        if self.word is True:
            self.word_vectorizer = CountVectorizer(ngram_range=(1,1))
            word_data, self.word_vectorizer = self._fit_transform(self.word_vectorizer)
            feature_list.append(word_data)
        
        ## Function Word counts ##
        if self.fnword is True:
            self.fnword_vectorizer = CountVectorizer(vocabulary=STOPWORDS)
            fnword_data, self.fnword_vectorizer = self._fit_transform(self.fnword_vectorizer)
            feature_list.append(fnword_data)

        print('lexdiv: %s char: %s pos: %s word: %s fnword: %s'%(self.lexdiv, self.char, self.pos, 
                                                                          self.word, self.fnword))
        feature_data = self.combine_features(tuple(feature_list)) 
        print('size of features:', feature_data.shape)   
        return feature_data
    
    def combine_features(self, feat_tuple):
        feature_data = hstack(feat_tuple)
        return feature_data
    
    def replace_words_POS(self, texts):
        return [' '.join(get_pos(text)) for text in texts]
    
    def lexical_diversity(self, docs):
        lex_div = np.array([len(set(text)) / len(text) for (text,label) in docs]).reshape(len(docs), 1)
        # print('lexical diversity:', lex_div.shape)
        return lex_div
    
    def _fit_transform(self, vectorizer, pos_replace=False):
        if pos_replace is True:
            texts = self.replace_words_POS(self.texts) # replace words in text with POS
        else:
            texts = self.texts
        vec_data = vectorizer.fit_transform(texts)
        vec_data = vec_data.astype(float)
        return vec_data, vectorizer
    
    def _transform(self, vectorizer, pos_replace=False):
        if pos_replace is True:
            texts = self.replace_words_POS(self.texts) # replace words in text with POS
        else:
            texts = self.texts
        vec_data = vectorizer.transform(texts)
        vec_data = vec_data.astype(float)
        return vec_data
    
    def transform(self, docs):
        feature_list = []
        self.texts = [text for i,(text,label) in enumerate(docs)]
        if self.lexdiv is True or not any((self.char, self.pos, self.word, self.fnword, self.lexdiv)):
            lex_div = self.lexical_diversity(docs)
            if not any((self.char, self.pos, self.word, self.fnword)):
                lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
            feature_list.append(lex_div)
        if self.char is True:
            feature_list.append(self._transform(self.char_vectorizer))
        if self.word is True:
            feature_list.append(self._transform(self.word_vectorizer))
        if self.pos is True:
            feature_list.append(self._transform(self.pos_vectorizer, pos_replace=True))
        if self.fnword is True:
            feature_list.append(self._transform(self.fnword_vectorizer))
        print('Extracting features from test data')
        print('lexdiv: %s char: %s pos: %s word: %s fnword: %s'%(self.lexdiv, self.char, self.pos, 
                                                                          self.word, self.fnword))
        feature_data = self.combine_features(tuple(feature_list)) 
        print('size of features:', feature_data.shape)   
        return feature_data
        
def write_results(path, problem, unk_folder, predictions):
    # Saving output data
    out_data=[]
    unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
    pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
    for i,v in enumerate(predictions):
        out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
    with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
        json.dump(out_data, f, indent=4)
    print('\t', 'answers saved to file','answers-'+problem+'.json')
    

def get_baseline_model_fn(num_in, num_out):
    # for Keras: initialize baseline model fn with num features and num predicted categories
    def baseline_model():
        # create model
        model = Sequential()
        model.add(Dense(8, input_dim=num_in, activation='relu'))
        model.add(Dense(num_out, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    return baseline_model

In [50]:

def pipeline_svm(path, outpath):
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    scores_all = []
    param_grids = [{
        'features__char': [True],
        'features__pos': [False],
        'features__word': [False],
        'features__lexdiv': [True],
        'features__fnword': [True],
        'sel__percentile': [70],
        'svm__C': [0.1],
        },
        {
        'features__char': [True],
        'features__pos': [True],
        'features__word': [False],
        'features__lexdiv': [True],
        'features__fnword': [True],
        'sel__percentile': [65],
        'svm__C': [10],
        }
    ]
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        svm = SVC(random_state=442)
        feat_extractor = Feature_Extractor()
        sel = SelectPercentile(f_classif)
        scaler = preprocessing.MaxAbsScaler()
        pipe = Pipeline(steps=[('features', feat_extractor), ('sel', sel), 
                               ('scale', scaler), ('svm', svm)])
        param_grid = param_grids[index]
        search = GridSearchCV(pipe, param_grid, iid=False, cv=5, n_jobs=3, verbose=10)
        search.fit(docs, train_labels)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        scores_all.append(search.cv_results_)
    return scores_all


%timeit

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
svm_results = pipeline_svm(base_dir,out_dir)
svm_results

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 out of   5 | elapsed:    2.5s remaining:    3.8s
[Parallel(n_jobs=3)]: Done   3 out of   5 | elapsed:    3.0s remaining:    2.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    4.9s finished


Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: False fnword: True
size of features: (140, 28650)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Best parameter (CV score=0.730):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': False, 'features__word': False, 'sel__percentile': 70, 'svm__C': 0.1}
problem00002
	 language:  en
	 5 candidate authors
	 35 known texts
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 out of   5 | elapsed:    5.1s remaining:    7.6s
[Parallel(n_jobs=3)]: Done   3 out of   5 | elapsed:    5.8s remaining:    3.8s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:   10.7s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:   10.7s finished


Extracting features from train data
char vocabulary size: 14415
	 pos vocabulary size: 619
lexdiv: True char: True pos: True word: False fnword: True
size of features: (35, 15214)
Best parameter (CV score=0.980):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': True, 'features__word': False, 'sel__percentile': 65, 'svm__C': 10}


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/dep

[{'mean_fit_time': array([1.5294313]),
  'std_fit_time': array([0.18031354]),
  'mean_score_time': array([0.13267465]),
  'std_score_time': array([0.04177476]),
  'param_features__char': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__fnword': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__lexdiv': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__pos': masked_array(data=[False],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__word': masked_array(data=[False],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_sel__percentile': masked_array(data=[70],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_svm__C': masked_

In [65]:

def pipeline_perceptron(path, outpath):
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    scores_all = []
    param_grids = [{
        'features__char': [True],
        'features__pos': [False],
        'features__word': [False],
        'features__lexdiv': [False],
        'features__fnword': [True],
        'sel__percentile': [55], # 0.59 score
        'clf__alpha': [ 1], # .84 score
        'clf__learning_rate_init': [ 0.001],
        },
        {
        'features__char': [True],
        'features__pos': [False],
        'features__word': [True],
        'features__lexdiv': [True],
        'features__fnword': [True],
        'sel__percentile': [55],  # 0.98 score
        'clf__alpha': [ 1],
        'clf__learning_rate_init': [0.001],
        }
    ]
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        clf = MLPClassifier(random_state=442)
        feat_extractor = Feature_Extractor()
        sel = SelectPercentile(f_classif)
        scaler = preprocessing.MaxAbsScaler()
        pipe = Pipeline(steps=[('features', feat_extractor), ('sel', sel), 
                               ('scale', scaler), ('clf', clf)])
        param_grid = param_grids[index]
        search = GridSearchCV(pipe, param_grid, iid=False, cv=5, n_jobs=3, verbose=10)
        search.fit(docs, train_labels)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        scores_all.append(search.cv_results_)
    return scores_all


%timeit

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
svm_results = pipeline_perceptron(base_dir,out_dir)
svm_results

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    5.0s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   16.1s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   23.8s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:   36.9s
[Parallel(n_jobs=3)]: Done  23 out of  25 | elapsed:  1.2min remaining:    6.5s
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:  1.5min finished


Extracting features from train data
char vocabulary size: 28470
lexdiv: False char: True pos: False word: False fnword: True
size of features: (140, 28649)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Best parameter (CV score=0.840):
{'clf__alpha': 1, 'clf__learning_rate_init': 0.001, 'features__char': True, 'features__fnword': True, 'features__lexdiv': False, 'features__pos': False, 'features__word': False, 'sel__percentile': 55}
problem00002
	 language:  en
	 5 candidate authors
	 35 known texts
Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

Extracting features from train data
char vocabulary size: 14415
lexdiv: True char: True pos: False word: True fnword: True
size of features: (35, 18878)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Best parameter (CV score=0.980):
{'clf__alpha': 1, 'clf__learning_rate_init': 0.001, 'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 55}


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

[{'mean_fit_time': array([ 5.2441958 ,  5.0670495 ,  4.90210514,  4.47331686, 28.07843208]),
  'std_fit_time': array([0.7838351 , 1.0539565 , 0.6981595 , 0.62890445, 3.08307084]),
  'mean_score_time': array([0.14665532, 0.1347311 , 0.14142408, 0.13235531, 0.16557746]),
  'std_score_time': array([0.0601638 , 0.06923629, 0.02966453, 0.04928331, 0.06388782]),
  'param_clf__alpha': masked_array(data=[0.0001, 0.001, 0.01, 0.1, 1],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_clf__learning_rate_init': masked_array(data=[0.001, 0.001, 0.001, 0.001, 0.001],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_features__char': masked_array(data=[True, True, True, True, True],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_features__fnword': masked_array(data=[True, True, True, True, 

In [66]:
pd.DataFrame(svm_results[0]).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_clf__learning_rate_init,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
4,28.078432,3.083071,0.165577,0.063888,1,0.001,True,True,False,False,...,0.840,0.075166,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,5.067050,1.053957,0.134731,0.069236,0.001,0.001,True,True,False,False,...,0.595,0.224944,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,5.244196,0.783835,0.146655,0.060164,0.0001,0.001,True,True,False,False,...,0.590,0.213073,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,4.473317,0.628904,0.132355,0.049283,0.1,0.001,True,True,False,False,...,0.580,0.207002,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,4.902105,0.698160,0.141424,0.029665,0.01,0.001,True,True,False,False,...,0.575,0.204939,5,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [67]:
pd.DataFrame(svm_results[1]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_clf__learning_rate_init,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
4,13.723784,3.435787,0.094811,0.050832,1,0.001,True,True,True,False,...,0.98,0.040000,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,1.870191,0.112287,0.077335,0.036382,0.0001,0.001,True,True,True,False,...,0.94,0.080000,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,1.857959,0.181203,0.085333,0.037960,0.001,0.001,True,True,True,False,...,0.92,0.097980,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,1.868897,0.166550,0.071390,0.028825,0.01,0.001,True,True,True,False,...,0.92,0.097980,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,15.546641,0.960631,0.075501,0.036019,0.1,0.001,True,True,True,False,...,0.92,0.074833,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [57]:
pd.DataFrame(svm_results[0]).to_csv('problem1-clfparams_perceptron.csv')
pd.DataFrame(svm_results[1]).to_csv('problem2-clfparams_perceptron.csv')

In [157]:
df_char_1 = pd.read_csv('problem1-charngrams.csv')
df_char_1range = pd.read_csv('problem1-charngramsrange.csv')
df_char_2 = pd.read_csv('problem2-charngrams.csv')
df_char_2range = pd.read_csv('problem2-charngramsrange.csv')
np.mean([pd.concat([df_char_1,df_char_1range])['mean_test_score'], pd.concat([df_char_2,df_char_2range])['mean_test_score']], axis=0)


array([0.565 , 0.63  , 0.7925, 0.8225, 0.785 , 0.7975, 0.835 , 0.825 ])

In [162]:
pd.concat([df_char_2,df_char_2range])['params'].values

array(["{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 1, 'features__pos': False, 'features__word': False}",
       "{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 2, 'features__pos': False, 'features__word': False}",
       "{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 3, 'features__pos': False, 'features__word': False}",
       "{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 4, 'features__pos': False, 'features__word': False}",
       "{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 5, 'features__pos': False, 'features__word': False}",
       "{'features__char': True, 'features__fnword': False, 'features__ft': 3, 'features__lexdiv': False, 'features__n': 3, '

In [33]:
def baseline_crossval(path, outpath, n=3, ft=5, feature_selection=False, 
             open_set=False, c=1, feat_sel_percent=None, classif=None, calibration=True, clf_params=[]):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    problem_scores = []
    scores_all = []
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        skf = StratifiedKFold(n_splits=5,random_state=442)
        scores = []
        k = 1
        for train_index, test_index in skf.split(docs, train_labels):
            print('Testing fold ', k)
            k +=1
            X_train_docs = [docs[i] for i in train_index]
            X_test_docs = [docs[i] for i in test_index]
            y_train, y_test = train_labels[train_index], train_labels[test_index]
            feat_extractor = Feature_Extractor(n, ft)
            X_train = feat_extractor.fit_transform(X_train_docs)
            X_test = feat_extractor.transform(X_test_docs)
            print('train shape:', X_train.shape, 'test shape:', X_test.shape)
            if feature_selection is True:
                ####### Feature Selection - Fit #######
                print("training before feature selection:", X_train.shape)
                print("testing before feature selection:", X_test.shape)
                #sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
                #train_data = sel.fit_transform(train_data)
                sel = SelectPercentile(f_classif, percentile=feat_sel_percent)
                X_train = sel.fit_transform(X_train, y_train)
                X_test = sel.transform(X_test)
                #sel = SelectKBest(chi2, k=100000)
                #train_data = sel.fit_transform(train_data, train_labels)
                print("training after feature selection:", X_train.shape)
                print("testing after feature selection:", X_test.shape)
            max_abs_scaler = preprocessing.MaxAbsScaler()
            X_train = max_abs_scaler.fit_transform(X_train)
            X_test = max_abs_scaler.transform(X_test)
            for c_params in clf_params:
                print('Fitting classifier with params:', c_params)
                if calibration is True:
                    clf=CalibratedClassifierCV(OneVsRestClassifier(classif(**c_params)))
                else:
                    clf=OneVsRestClassifier(classif(**clf_params))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='macro')
                recall = recall_score(y_test, y_pred, average='macro')
                f1 = f1_score(y_test, y_pred, average='macro')
                fold_scores = {'problem': problem, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 
                               'f1': f1, **c_params}
                print(fold_scores)
                scores.append(fold_scores)
                #write_results(path, problem, unk_folder, y_pref)
        '''
        mean_accuracy = np.mean([x[0] for x in scores])
        print(problem, 'MEAN ACCURACY SCORES:', mean_accuracy)
        mean_prec = np.mean([x[1] for x in scores])
        print(problem, 'MEAN PRECISION SCORES:', mean_prec)
        mean_recall = np.mean([x[2] for x in scores])
        print(problem, 'MEAN RECALL SCORES:', mean_recall)
        mean_f1 = np.mean([x[3] for x in scores])
        print(problem, 'MEAN F1 SCORES:', mean_f1)
        problem_scores.append(mean_accuracy)
        '''
        scores_all.extend(scores)
    #print('MEAN SCORES ACROSS PROBLEMS:', np.mean(problem_scores))
    # todo: also add stdev of scores
    print('elapsed time:', time.time() - start_time)
    return pd.DataFrame(scores_all)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
#params = {'n': 5,'ft': 3, 'feature_selection': True, 'clf_params': [{'C': 0.1},{'C': 1}], 'feat_sel_percent': 85, 'classif': SVC, 'calibration': True}
params = {'n': 5,'ft': 3, 'feature_selection': True, 'clf_params': [{}], 'feat_sel_percent': 85, 'classif': AdaBoostClassifier, 'calibration': True}


#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'clf_params': [{'C': 0.1, 'solver': 'lbfgs','multi_class': 'multinomial','max_iter': 500}], 'feat_sel_percent': 85, 'classif': LogisticRegression, 'calibration': True}

#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'clf_params': {'hidden_layer_sizes':(50,), 'max_iter':10, 'alpha':1e-4,
#                    'solver':'sgd', 'verbose':10, 'tol':1e-4, 'random_state':1, 'learning_rate_init':.1},
#          'feat_sel_percent': 85, 'classif': MLPClassifier, 'calibration': True}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'clf_params': {'hidden_layer_sizes':(1,), 'random_state':1, 'learning_rate_init':.1},
#          'feat_sel_percent': 85, 'classif': MLPClassifier, 'calibration': True}
%timeit
df = baseline_crossval(base_dir,out_dir,**params)
df

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Testing fold  1
(100, 52773)
	 pos vocabulary size: 806 char vocabulary size: 52773
[ 21.  25.   6.  37.  39.  52.  16.  48.  59.  49.  99.  85.  47.  55.
  49.  50.  37.  31.  42.  61.  87.  31.  40.  24.  59.  46.  43.  68.
  47.  85.  42. 120.  94. 110. 127. 107.  31.  59. 108.  95.  77.  80.
  75. 118.  98.  32.  54.  46.  64.  46.  54.  47.  86.  52.  68.  78.
  57.  77.  90.  77.  62.  54.  55.  74.  86.  55.  55.  34.  61.  28.
  70.  57.  44.  40.  66.  16.  13.  42.  21.  68.  62.  42. 105.  84.
  38.  92.  93.  91.  87. 107.  45.  88.  57. 103.  65.  67.  38.  49.
  73.  76.]
pos data: (100, 806) char data: (100, 52773) word data: (100, 116893)
train shape: (100, 170473) test shape: (40, 170473)
training before feature selection: (100, 170473)
testing before feature selection: (40, 170473)
training after feature selection: (100, 144902)
testing after feature selection: (40, 144902)
Fitting classifier with p

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no

{'problem': 'problem00001', 'accuracy': 0.175, 'precision': 0.16666666666666669, 'recall': 0.175, 'f1': 0.16333333333333333}
Testing fold  2
(100, 52929)
	 pos vocabulary size: 807 char vocabulary size: 52929
[ 63.  80.  74.  79.  72.  64.  76.  87.  74.  74. 101. 104. 103. 101.
  95.  87.  79. 102. 103.  80.  77. 111. 115.  82. 111.  39.  45.  73.
  73.  59.  92.  59.  73.  81.  76.  75.  72.  95.  85.  63.  66.  74.
  89.  81.  67. 100.  75.  89.  78. 100.  79.  82.  73.  97.  71.  86.
  85.  97.  87. 100. 100.  55.  62.  68.  74.  98.  84. 103.  74. 103.
  98. 116.  62.  86.  78.  68.  73.  49.  69.  81.  58.  74.  75.  76.
  63.  90. 101.  84.  94.  83.  80.  83.  61.  80.  88.  64.  78. 109.
  71.  87.]
pos data: (100, 807) char data: (100, 52929) word data: (100, 116940)
train shape: (100, 170677) test shape: (40, 170677)
training before feature selection: (100, 170677)
testing before feature selection: (40, 170677)
training after feature selection: (100, 144495)
testing after fe

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no

{'problem': 'problem00001', 'accuracy': 0.175, 'precision': 0.12944444444444442, 'recall': 0.175, 'f1': 0.13080086580086578}
Testing fold  3
(120, 58556)
	 pos vocabulary size: 826 char vocabulary size: 58556
[ 63.  80.  70.  71.  79.  72.  64.  76.  97.  98.  74.  74. 101. 104.
 106. 112. 101.  95.  87.  79.  64.  69. 103.  80.  77. 111.  87.  85.
  82. 111.  39.  45.  72.  61.  73.  59.  92.  59.  78.  64.  81.  76.
  75.  72.  95.  67.  85.  63.  66.  74.  84.  73.  81.  67. 100.  75.
  62.  49.  78. 100.  79.  82.  69.  89.  97.  71.  86.  85. 104.  99.
  87. 100. 100.  55.  75.  67.  68.  74.  98.  84.  91.  96.  74. 103.
  98. 116.  56.  77.  86.  78.  68.  73.  70.  76.  69.  81.  58.  74.
  66.  87.  76.  63.  90. 101.  77.  79.  94.  83.  80.  83.  56.  85.
  80.  88.  64.  78.  84.  95.  71.  87.]
pos data: (120, 826) char data: (120, 58556) word data: (120, 137457)
train shape: (120, 196840) test shape: (20, 196840)
training before feature selection: (120, 196840)
testing be

...

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no

{'problem': 'problem00002', 'accuracy': 0.6, 'precision': 0.4666666666666667, 'recall': 0.6, 'f1': 0.5}
Testing fold  5
(30, 24084)
	 pos vocabulary size: 601 char vocabulary size: 24084
[29. 40. 36. 24. 41. 37. 12. 28. 19. 12. 11. 25. 40. 17. 39. 13. 43. 16.
 33. 18. 40. 43. 31. 38. 28. 25. 20. 33. 18. 39.]
pos data: (30, 601) char data: (30, 24084) word data: (30, 38670)
train shape: (30, 63356) test shape: (5, 63356)
training before feature selection: (30, 63356)
testing before feature selection: (5, 63356)
training after feature selection: (30, 53852)
testing after feature selection: (5, 53852)
Fitting classifier with params: {}


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'problem': 'problem00002', 'accuracy': 0.6, 'precision': 0.6, 'recall': 0.6, 'f1': 0.6}
elapsed time: 527.2663791179657


,accuracy,f1,precision,problem,recall
0,0.175,0.163333,0.166667,problem00001,0.175
1,0.175,0.130801,0.129444,problem00001,0.175
2,0.250,0.166667,0.133333,problem00001,0.250
3,0.150,0.116667,0.100000,problem00001,0.150
4,0.300,0.233333,0.210000,problem00001,0.300
5,0.500,0.526667,0.633333,problem00002,0.500
6,0.700,0.633333,0.600000,problem00002,0.700
7,0.600,0.600000,0.600000,problem00002,0.600
8,0.600,0.500000,0.466667,problem00002,0.600
9,0.600,0.600000,0.600000,problem00002,0.600


In [75]:
df.groupby('problem')['f1'].mean()

problem
problem00001    0.16216
problem00002    0.57200
Name: f1, dtype: float64

In [30]:
def baseline_keras(path, outpath, n=3, ft=5, pt=0.1, feature_selection=False, 
             open_set=False, c=1, feat_sel_percent=None, clf=None, calibration=True):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    problem_scores = []
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        if calibration is True:
            clf=CalibratedClassifierCV(OneVsRestClassifier(SVC(C=c)))
        else:
            clf=OneVsRestClassifier(SVC(C=c))
        skf = StratifiedKFold(n_splits=5,random_state=442)
        scores = []
        print("Do cross validation.")
        for train_index, test_index in skf.split(docs, train_labels):
            X_train_docs = [docs[i] for i in train_index]
            X_test_docs = [docs[i] for i in test_index]
            y_train, y_test = train_labels[train_index], train_labels[test_index]
            ##### Extract X features ####
            feat_extractor = Feature_Extractor(n, ft)
            X_train = feat_extractor.fit_transform(X_train_docs)
            X_test = feat_extractor.transform(X_test_docs)
            print('train shape:', X_train.shape, 'test shape:', X_test.shape)
            if feature_selection is True:
                ####### Feature Selection - Fit #######
                print("training before feature selection:", X_train.shape)
                print("testing before feature selection:", X_test.shape)
                #sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
                #train_data = sel.fit_transform(train_data)
                # We use the default selection function: the 10% most significant features
                sel = SelectPercentile(f_classif, percentile=feat_sel_percent)
                X_train = sel.fit_transform(X_train, y_train)
                X_test = sel.transform(X_test)
                #sel = SelectKBest(chi2, k=100000)
                #train_data = sel.fit_transform(train_data, train_labels)
                print("training after feature selection:", X_train.shape)
                print("testing after feature selection:", X_test.shape)
            max_abs_scaler = preprocessing.MaxAbsScaler()
            X_train = max_abs_scaler.fit_transform(X_train)
            X_test = max_abs_scaler.transform(X_test)
            baseline_model = get_baseline_model_fn(X_train.shape[1], len(candidates))
            clf = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
            ##### Convert y to 1-hot encoding ####
            y_encoder = LabelEncoder()
            y_train = y_encoder.fit_transform(y_train)
            # convert integers to dummy variables (i.e. one hot encoded)
            y_train = np_utils.to_categorical(y_train)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print("y_pred y_test:", y_pred.shape, y_test.shape)
            y_pred = y_encoder.inverse_transform(y_pred)
            print(y_pred)
            print(y_test)
            accuracy = accuracy_score(y_test, y_pred)
            scores.append(accuracy)
            print('accuracy:', accuracy)
            #write_results(path, problem, unk_folder, predictions)
        mean_score = np.mean(scores)
        print(problem, 'MEAN ACCURACY SCORES:', mean_score)
        problem_scores.append(mean_score)
    print('MEAN SCORES ACCROSS PROBLEMS:', np.mean(problem_scores))
    # todo: also add stdev of scores
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': False, 'c':0.1, 'feat_sel_percent': None, 'clf': 'SVC'}
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'SVC', 'calibration': True}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':10, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline_keras(base_dir,out_dir,**params)
%timeit
    

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Do cross validation.
(100, 52773)
	 pos vocabulary size: 806 char vocabulary size: 52773
lexical diversity: (100, 1)
[ 21.  25.   6.  37.  39.  52.  16.  48.  59.  49.  99.  85.  47.  55.
  49.  50.  37.  31.  42.  61.  87.  31.  40.  24.  59.  46.  43.  68.
  47.  85.  42. 120.  94. 110. 127. 107.  31.  59. 108.  95.  77.  80.
  75. 118.  98.  32.  54.  46.  64.  46.  54.  47.  86.  52.  68.  78.
  57.  77.  90.  77.  62.  54.  55.  74.  86.  55.  55.  34.  61.  28.
  70.  57.  44.  40.  66.  16.  13.  42.  21.  68.  62.  42. 105.  84.
  38.  92.  93.  91.  87. 107.  45.  88.  57. 103.  65.  67.  38.  49.
  73.  76.]
pos data: (100, 806) char data: (100, 52773) word data: (100, 116893)
lexical diversity: (40, 1)
train shape: (100, 170473) test shape: (40, 170473)
training before feature selection: (100, 170473)
testing before feature selection: (40, 170473)
training after feature selection: (100, 144902)
testing aft

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (40,) (40,)
['candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00012' 'candidate00012' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013']
['candidate00001' 'candidate00001' 'candidate00002' 'candidate00002'
 'candidate00003' 'candidate00003' 'candidate00004' 'candidate00004'
 'candidate00005' 'candidate00005' 'candidate00006' 'candidate00006'
 'candidate00007' 'candidate00007' 'candidate00008' 'candidate00008'
 'cand

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (40,) (40,)
['candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00014' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005']
['candidate00001' 'candidate00001' 'candidate00002' 'candidate00002'
 'candidate00003' 'candidate00003' 'candidate00004' 'candidate00004'
 'candidate00005' 'candidate00005' 'candidate00006' 'candidate00006'
 'candidate00007' 'candidate00007' 'candidate00008' 'candidate00008'
 'cand

...

y_pred y_test: (5,) (5,)
['candidate00001' 'candidate00005' 'candidate00003' 'candidate00001'
 'candidate00005']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.6
(30, 23884)
	 pos vocabulary size: 592 char vocabulary size: 23884
lexical diversity: (30, 1)
[29. 40. 36. 24. 41. 29. 12. 28. 19. 12. 11. 12. 40. 17. 39. 13. 43. 31.
 33. 18. 40. 43. 31. 47. 28. 25. 20. 33. 18. 24.]
pos data: (30, 592) char data: (30, 23884) word data: (30, 38443)
lexical diversity: (5, 1)
train shape: (30, 62920) test shape: (5, 62920)
training before feature selection: (30, 62920)
testing before feature selection: (5, 62920)
training after feature selection: (30, 53482)
testing after feature selection: (5, 53482)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (5,) (5,)
['candidate00004' 'candidate00004' 'candidate00004' 'candidate00004'
 'candidate00004']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.2
(30, 24084)
	 pos vocabulary size: 601 char vocabulary size: 24084
lexical diversity: (30, 1)
[29. 40. 36. 24. 41. 37. 12. 28. 19. 12. 11. 25. 40. 17. 39. 13. 43. 16.
 33. 18. 40. 43. 31. 38. 28. 25. 20. 33. 18. 39.]
pos data: (30, 601) char data: (30, 24084) word data: (30, 38670)
lexical diversity: (5, 1)
train shape: (30, 63356) test shape: (5, 63356)
training before feature selection: (30, 63356)
testing before feature selection: (5, 63356)
training after feature selection: (30, 53852)
testing after feature selection: (5, 53852)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (5,) (5,)
['candidate00001' 'candidate00003' 'candidate00003' 'candidate00001'
 'candidate00005']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.6
problem00002 MEAN ACCURACY SCORES: 0.43999999999999995
MEAN SCORES ACCROSS PROBLEMS: 0.24499999999999997
elapsed time: 484.9901819229126


In [ ]:
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': False, 'c':0.1, 'feat_sel_percent': None, 'clf': 'SVC'}
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'SVC', 'calibration': True}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':10, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline(base_dir,out_dir,**params)
%timeit

In [76]:
def baseline_old(path, outpath, n=3, ft=5, pt=0.1, feature_selection=False, 
             open_set=False, c=1, feat_sel_percent=80, clf=None):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        train_docs=[]
        for candidate in candidates:
            train_docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = [label for i,(text,label) in enumerate(train_docs)]
        #### Feature Extraction ###
        ###### Fit-Transform Training Set #######
        feat_extractor = Feature_Extractor(n, ft)
        train_data = feat_extractor.fit_transform(train_docs)
        if feature_selection is True:
            ####### Feature Selection - Fit #######
            print("training before feature selection:", train_data.shape)
            #sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
            #train_data = sel.fit_transform(train_data)
            # We use the default selection function: the 10% most significant features
            sel = SelectPercentile(f_classif, percentile=feat_sel_percent)
            train_data = sel.fit_transform(train_data, train_labels)
            #sel = SelectKBest(chi2, k=100000)
            #train_data = sel.fit_transform(train_data, train_labels)
            print("training after feature selection:", train_data.shape)
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(train_docs), 'known texts')
        
        ###### Transform Test Set #######
        test_docs=read_files(path+os.sep+problem,unk_folder)
        test_data = feat_extractor.transform(test_docs)
        if feature_selection is True:
            ####### Feature Selection #######
            print("test before feature selection:", test_data.shape)
            test_data = sel.transform(test_data)
            print("test after feature selection:", test_data.shape)
        print('\t', len(test_docs), 'unknown texts')
        
        ###### Applying Classifiers #####
        max_abs_scaler = preprocessing.MaxAbsScaler()
        scaled_train_data = max_abs_scaler.fit_transform(train_data)
        scaled_test_data = max_abs_scaler.transform(test_data)
        clf=CalibratedClassifierCV(OneVsRestClassifier(SVC(C=c)))
        clf.fit(scaled_train_data, train_labels)
        predictions=clf.predict(scaled_test_data)
        if open_set is True:
            # Reject option (used in open-set cases)
            count=0
            for i,p in enumerate(predictions):
                sproba=sorted(proba[i],reverse=True)
                if sproba[0]-sproba[1]<pt:
                    predictions[i]=u'<UNK>'
                    count=count+1
            print('\t',count,'texts left unattributed')
        # Saving output data
        out_data=[]
        unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
        pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
        for i,v in enumerate(predictions):
            out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
        with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
            json.dump(out_data, f, indent=4)
        print('\t', 'answers saved to file','answers-'+problem+'.json')
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline_old(base_dir,out_dir,**params)
%timeit

problem00001
(140, 64239)
	 pos vocabulary size: 861 char vocabulary size: 64239
pos data: (140, 861) char data: (140, 64239) word data: (140, 158555) fnword data: (140, 179)
training before feature selection: (140, 223835)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


training after feature selection: (140, 190259)
	 language:  en
	 20 candidate authors
	 140 known texts
test before feature selection: (105, 223835)
test after feature selection: (105, 190259)
	 105 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change fro

	 answers saved to file answers-problem00001.json
problem00002
(35, 26761)
	 pos vocabulary size: 619 char vocabulary size: 26761
pos data: (35, 619) char data: (35, 26761) word data: (35, 44504) fnword data: (35, 179)
training before feature selection: (35, 72064)
training after feature selection: (35, 56984)
	 language:  en
	 5 candidate authors
	 35 known texts


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


test before feature selection: (21, 72064)
test after feature selection: (21, 56984)
	 21 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change fro

	 answers saved to file answers-problem00002.json
elapsed time: 41.05219507217407


In [31]:
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'Keras Neural Network'}

In [ ]:
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'Keras Neural Network'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': False, 'c':0.1, 'feat_sel_percent': None, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline_old(base_dir,out_dir,**params)
%timeit

In [77]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
# Evaluation script for the Cross-Domain Authorship Attribution task @PAN2019.
We use the F1 metric (macro-average) as implemented in scikit-learn:
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
We include the following ad hoc rules:
- If authors are predicted which were not seen during training,
  these predictions will count as false predictions ('<UNK>' class)
  and they will negatively effect performance.
- If texts are left unattributed they will assigned to the ('<UNK>'
  class) and they will negatively effect performance.
- The <UNK> class is excluded from the macro-average across classes.
- If multiple test attributions are given for a single unknown document,
  only the first one will be taken into consideration.

Dependencies:
- Python 2.7 or 3.6 (we recommend the Anaconda Python distribution)
- scikit-learn

Usage from the command line:
>>> python pan19-cdaa-evaluator.py -i COLLECTION -a ANSWERS -o OUTPUT
where
    COLLECTION is the path to the main folder of the evaluation collection
    ANSWERS is the path to the answers folder of a submitted method
    OUTPUT is the path to the folder where the results of the evaluation will be saved

Example: 
>>>  python pan19-cdaa-evaluator.py -i ".\pan19-cross-domain-authorship-attribution-training-dataset-2019-01-23\" -a ".\answ
ers-unigram" -o ".\eval-unigram\"

# References:
@article{scikit-learn,
 title={Scikit-learn: Machine Learning in {P}ython},
 author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
         and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
         and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
         Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
 journal={Journal of Machine Learning Research},
 volume={12},
 pages={2825--2830},
 year={2011}
}
"""

import argparse
import os
import json
import warnings
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

def eval_measures(gt, pred):
    """Compute macro-averaged F1-scores, macro-averaged precision, 
    macro-averaged recall, and micro-averaged accuracy according the ad hoc
    rules discussed at the top of this file.
    Parameters
    ----------
    gt : dict
        Ground truth, where keys indicate text file names
        (e.g. `unknown00002.txt`), and values represent
        author labels (e.g. `candidate00003`)
    pred : dict
        Predicted attribution, where keys indicate text file names
        (e.g. `unknown00002.txt`), and values represent
        author labels (e.g. `candidate00003`)
    Returns
    -------
    f1 : float
        Macro-averaged F1-score
    precision : float
        Macro-averaged precision
    recall : float
        Macro-averaged recall
    accuracy : float
        Micro-averaged F1-score
    """

    actual_authors = list(gt.values())
    encoder = LabelEncoder().fit(['<UNK>'] + actual_authors)

    text_ids, gold_authors, silver_authors = [], [], []
    for text_id in sorted(gt):
        text_ids.append(text_id)
        gold_authors.append(gt[text_id])
        try:
            silver_authors.append(pred[text_id])
        except KeyError:
            # missing attributions get <UNK>:
            silver_authors.append('<UNK>')

    assert len(text_ids) == len(gold_authors)
    assert len(text_ids) == len(silver_authors)

    # replace non-existent silver authors with '<UNK>':
    silver_authors = [a if a in encoder.classes_ else '<UNK>' 
                      for a in silver_authors]

    gold_author_ints = encoder.transform(gold_authors)
    silver_author_ints = encoder.transform(silver_authors)

    # get F1 for individual classes (and suppress warnings):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        labels=list(set(gold_author_ints))
        # Exclude the <UNK> class
        for x in labels:
            if encoder.inverse_transform(np.array([x]))=='<UNK>':
                labels.remove(x)
        f1 = f1_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        precision = precision_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        recall = recall_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        accuracy = accuracy_score(gold_author_ints,
                  silver_author_ints)

    return f1,precision,recall

def evaluate(ground_truth_file,predictions_file):
    # Calculates evaluation measures for a single attribution problem
    gt = {}
    with open(ground_truth_file, 'r') as f:
        for attrib in json.load(f)['ground_truth']:
            gt[attrib['unknown-text']] = attrib['true-author']

    pred = {}
    with open(predictions_file, 'r') as f:
        for attrib in json.load(f):
            if attrib['unknown-text'] not in pred:
                pred[attrib['unknown-text']] = attrib['predicted-author']
    f1,precision,recall =  eval_measures(gt,pred)
    return round(f1,3), round(precision,3), round(recall,3)

def evaluate_all(path_collection,path_answers,path_out,params):
    # Calculates evaluation measures for a PAN-18 collection of attribution problems
    infocollection = path_collection+os.sep+'collection-info.json'
    problems = []
    data = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
    scores=[];
    for problem in problems:
        prob_data = deepcopy(params)
        f1,precision,recall=evaluate(path_collection+os.sep+problem+os.sep+'ground-truth.json',path_answers+os.sep+'answers-'+problem+'.json')
        scores.append(f1)
        prob_data.update({'problem-name': problem, 'macro-f1': round(f1,3), 'macro-precision': round(precision,3), 'macro-recall': round(recall,3)})
        if os.path.isfile('metrics.csv'):
            with open('metrics.csv', 'a') as f:  # Just use 'w' mode in 3.x
                w = csv.DictWriter(f, prob_data.keys())
                w.writerow(prob_data)
        else:
            with open('metrics.csv', 'w') as f:  # Just use 'w' mode in 3.x
                w = csv.DictWriter(f, prob_data.keys())
                w.writeheader()
                w.writerow(prob_data)
        data.append(prob_data)
        print(str(problem),'Macro-F1:',round(f1,3))
    overall_score=sum(scores)/len(scores)
    # Saving data to output files (out.json and evaluation.prototext)
    with open(path_out+os.sep+'out.json', 'w') as f:
        json.dump({'problems': data, 'overall_score': round(overall_score,3)}, f, indent=4, sort_keys=True)
    print('Overall score:', round(overall_score,3))
    prototext='measure {\n key: "mean macro-f1"\n value: "'+str(round(overall_score,3))+'"\n}\n'
    with open(path_out+os.sep+'evaluation.prototext', 'w') as f:
        f.write(prototext)
    return pd.read_csv('metrics.csv')
        
evaluate_all(base_dir,out_dir,eval_dir, params)


problem00001 Macro-F1: 0.585
problem00002 Macro-F1: 0.734
Overall score: 0.66


,n,ft,pt,feature_selection,c,feat_sel_percent,clf,problem-name,macro-f1,macro-precision,macro-recall
0,5,3,0.05,True,0.1,85,SVC,problem00001,0.573,0.578,0.679
1,5,3,0.05,True,0.1,85,SVC,problem00002,0.734,0.715,0.767
2,5,3,0.05,True,0.1,85,SVC,problem00001,0.585,0.582,0.713
3,5,3,0.05,True,0.1,85,SVC,problem00002,0.734,0.715,0.767
4,5,3,0.05,True,0.1,85,SVC,problem00001,0.573,0.578,0.679
5,5,3,0.05,True,0.1,85,SVC,problem00002,0.734,0.715,0.767
6,5,3,0.05,True,0.1,85,SVC,problem00001,0.585,0.582,0.713
7,5,3,0.05,True,0.1,85,SVC,problem00002,0.734,0.715,0.767


In [ ]:
df = pd.read_csv('metrics.csv')
df

In [ ]:
df.set_index('c', inplace=True)
df

In [ ]:
df[df['feature_selection'] == True].groupby(['problem-name'])['macro-recall', 'macro-precision'].plot(legend=True)